In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

# configuring the virtual module

In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-22 21:19:26,440 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 21:19:26,442 - settings - Setting database.user to celiib
INFO - 2020-11-22 21:19:26,443 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 21:19:26,449 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-22 21:19:26,450 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 21:19:26,462 - connection - Connect

Sleeping 17 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-22 21:19:26,654 - settings - Setting enable_python_native_blobs to True


# Defining Our Table

In [3]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [4]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 3246


In [5]:
#so that it will have the adapter defined
from datajoint_utils import *

In [ ]:
#minnie.Decomposition.drop()
#schema.external['decomposition'].delete(delete_external_files=True)

In [ ]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branche

    
    """

    key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj())
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Cleaning the external files

In [ ]:
(minnie.schema.jobs & "table_name='__decomposition'").delete()
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

In [ ]:
import preprocessing_vp2 as pre
import compartment_utils as cu
cu = reload(cu)
pre = reload(pre)

In [ ]:
import time
import random

start_time = time.time()
#time.sleep(random.randint(0, 900))
print('Populate Started')
Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')
print('Populate Done')

print(f"Total time for Decomposition populate = {time.time() - start_time}")

# Clearing Files

In [ ]:
curr_path = Path("/mnt/dj-stor01/platinum/minnie65/02/decomposition/")
for f in curr_path.iterdir():
    print(f'{f}')
    #f.unlink()
    

# Checking the Table

In [ ]:
minnie.Decomposition()

In [ ]:
du = reload(du)
from datajoint_utils import *
my_dj_neuron = (minnie.Decomposition & "segment_id=864691134221889045").fetch1("decomposition")

import neuron_visualizations as nviz
nviz.visualize_neuron(my_dj_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")

# Debugging the neuron creation

In [6]:
#1) Get the segment id from the key
segment_id = 864691136004787658
description = "0_25"
print(f"\n\n----- Working on {segment_id}-------")
global_start = time.time()

#2) Get the decimated mesh
current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

#3) Get the somas info *************************** Need to change this when actually run *******************
somas = du.get_soma_mesh_list(segment_id) 
print(f"somas = {somas}")
#4) Run the preprocessing


total_neuron_process_time = time.time()

INFO - 2020-11-22 21:19:35,899 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 21:19:36,056 - settings - Setting enable_python_native_blobs to True




----- Working on 864691136004787658-------


INFO - 2020-11-22 21:19:48,440 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 21:19:48,605 - settings - Setting enable_python_native_blobs to True


somas = [[<trimesh.Trimesh(vertices.shape=(19114, 3), faces.shape=(35733, 3))>, <trimesh.Trimesh(vertices.shape=(21229, 3), faces.shape=(41798, 3))>], array([388.2821, 388.2821]), array([0.341, 0.788])]


In [7]:
import preprocessing_vp2 as pre
import soma_extraction_utils as sm

import neuron
tu = reload(tu)
du = reload(du)
sm = reload(sm)
pre = reload(pre)
import compartment_utils as cu
cu = reload(cu)

neuron = reload(neuron)
print(f"\n--- Beginning preprocessing of {segment_id}---")
recovered_neuron = neuron.Neuron(
mesh = current_neuron_mesh,
somas = somas,
segment_id=segment_id,
description=description,
suppress_preprocessing_print=False,
suppress_output=False,
calculate_spines=True,
widths_to_calculate=["no_spine_median_mesh_center"]

        )

INFO - 2020-11-22 21:19:49,446 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 21:19:49,447 - settings - Setting database.user to celiib
INFO - 2020-11-22 21:19:49,448 - settings - Setting database.password to newceliipass



--- Beginning preprocessing of 864691136004787658---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(19114, 3), faces.shape=(35733, 3))>, <trimesh.Trimesh(vertices.shape=(21229, 3), faces.shape=(41798, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(19114, 3), faces.shape=(35733, 3))>, <trimesh.Trimesh(vertices.shape=(21229, 3), faces.shape=(41798, 3))>]
soma_mesh_list_centers = [array([ 513752.7642618 ,  530181.6500994 , 1111548.38427331]), array([447289.79102643, 827804.73290782, 991554.05846248])]
xvfb-run -n 3246 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging/temp/neuron_11077.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging/temp/neuron_11077_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging/

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:06<00:00,  6.09s/it]

Time for 1st pass MP skeletonization: 7.561906099319458
branches_touching_root = [19]
length of Graph = 23065
Working on path [4265. 4274. 4278. 4280.]
path_degrees = [3, 2, 2, 3]
Working on path [7221. 7240. 7278. 7300. 7311. 7324. 7338.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [7751. 7769. 7789. 7807. 7831. 7860.]
path_degrees = [3, 2, 2, 2, 2, 3]
Working on path [23067.  7920.  7960.  7978.]
path_degrees = [4, 2, 2, 3]
Working on path [20283. 20320.]
path_degrees = [3, 3]
Working on path [21786. 21787. 21788. 21789.]
path_degrees = [3, 2, 2, 3]
Working on path [22751. 22754.]
path_degrees = [3, 3]
max(kept_branches_idx) = 116, len(kept_branches_idx) = 110
empty_indices % = 0.0
 conflict_indices % = 0.015025801036054566


...


Deleting branches from dictionary
MP_branches_with_stitch_point= [0]
MP_branches_for_correspondence = [0]
MP_leftover = [], MP_leftover_idx = []
 Finished with (0, 0) 



Time for decomp of Limb = 18.592309713363647
Number of matching vertices = 18
Only one endpoint after filtering away the endpoints that are not on the skeleton
Total time for Skeletonization and Mesh Correspondence = 471.7720456123352


 ----- Working on Stitching ----------
Status of Main limb stitch point moved = True
Total time for stitching floating pieces = 0.39274168014526367


---------Working on soma_idx = 1, soma_group_idx 0, endpt = [ 509947.22894737  537027.36315789 1113273.18421053]---------
Starting_edge inside branches_to_conept = [[ 508842.          538142.         1113080.        ]
 [ 509947.22894737  537027.36315789 1113273.18421053]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [24]
printing out current edge:
[[ 509947.22894737  537027.36315789 111


There was only one mesh found from the spine process and mesh split, returning empty array



There was only one mesh found from the spine process and mesh split, returning empty array



There was only one mesh found from the spine process and mesh split, returning empty array



Working on limb L0 branch 2
No spines and using precomputed width
Working on limb L0 branch 1
Working on limb L0 branch 5
Working on limb L0 branch 6
No spines and using precomputed width
Working on limb L0 branch 104
Working on limb L0 branch 4
No spines and using precomputed width
Working on limb L0 branch 105
Working on limb L0 branch 18
Working on limb L0 branch 103
Working on limb L0 branch 3
No spines and using precomputed width
Working on limb L0 branch 20
Working on limb L0 branch 99
Working on limb L0 branch 102
Working on limb L0 branch 7
No spines and using precomputed width
Working on limb L0 branch 19
Working on limb L0 branch 21
Working on limb L0 branch 28
No spines and using precomputed width
Working on limb L0 branch 100
No spines and using precomputed width
Working on limb L0 branch 101
No spines and using precomputed width
Working on limb L0 branch 0
Working on limb L0 branch 106
Working on limb L0 branch 22
Working on limb L0 branch 30
No spines and using precomput

In [8]:
import neuron_visualizations as nviz

In [12]:
nviz.visualize_neuron(recovered_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton
Working on  new stand alone scatter points


In [ ]:
minnie.